#   Graph RAG, Vector RAG, Hybrid RAG

To enable Cognitive intelligence App towards private data, RAG + LLM and Knowledge Graph are state-of-the-art approaches.

In this demo, we will explain know-how of four types of approaches and compare the trade-off and performance among them.

| QueryEngine | Graph RAG query engine                                       | Vector RAG query engine                                      | Hybrid RAG query engine                                                                                          |
| ----------- | ------------------------------------------------------------ | ------------------------------------------------------------ |------------------------------------------------------------------------------------------------------------------|
| Mechanism   | 1. Get related entities of the question<br />2. Get n-depth **SubGraphs** of related entities from KG<br />3. Answer synthesis based on related SubGraphs | 1. Create embedding of question<br />2. Semantic search **top-k related doc chunks**<br />3. Answer synthesis based on related doc chunks | 1. Do retrieval as Vector and Graph RAG <br />2. Answer synthesis based on **both related chunks and SubGraphs** |

## Background, RAG

Below digrams are showing how RAG works:

```
                  RAG with Llama Index
                  ┌────┬────┬────┬────┐                  
                  │ 1  │ 2  │ 3  │ 4  │                  
                  ├────┴────┴────┴────┤                  
                  │  Docs/Knowledge   │                  
┌───────┐         │        ...        │       ┌─────────┐
│       │         ├────┬────┬────┬────┤       │         │
│       │         │ 95 │ 96 │    │    │       │         │
│       │         └────┴────┴────┴────┘       │         │
│ User  │─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─▶   LLM   │
│       │                                     │         │
│       │                                     │         │
└───────┘    ┌ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ┐  └─────────┘
    │          ┌──────────────────────────┐        ▲     
    └────────┼▶│  Tell me ....., please   │├───────┘     
               └──────────────────────────┘              
             │ ┌────┐ ┌────┐               │             
               │ 3  │ │ 96 │                             
             │ └────┘ └────┘               │             
              ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ 
```

In VectorDB based RAG, we create embeddings of each node(chunk), and find TopK related ones towards a given question during the query. In the above diagram, nodes `3` and `96` were fetched as the TopK related nodes, used to help answer the user query. 

## Background Graph RAG

In Graph RAG, we will extract relationships between entities, representing concise facts from each node. It would look something like this:

```
Node Split and Embedding

┌────┬────┬────┬────┐
│ 1  │ 2  │ 3  │ 4  │
├────┴────┴────┴────┤
│  Docs/Knowledge   │
│        ...        │
├────┬────┬────┬────┤
│ 95 │ 96 │    │    │
└────┴────┴────┴────┘
```

Then, if we zoomed in of it:

```
       Node Split and Embedding, with Knowledge Graph being extracted

┌──────────────────┬──────────────────┬──────────────────┬──────────────────┐
│ .─.       .─.    │  .─.       .─.   │            .─.   │  .─.       .─.   │
│( x )─────▶ y )   │ ( x )─────▶ a )  │           ( j )  │ ( m )◀────( x )  │
│ `▲'       `─'    │  `─'       `─'   │            `─'   │  `─'       `─'   │
│  │     1         │        2         │        3    │    │        4         │
│ .─.              │                  │            .▼.   │                  │
│( z )─────────────┼──────────────────┼──────────▶( i )─┐│                  │
│ `◀────┐          │                  │            `─'  ││                  │
├───────┼──────────┴──────────────────┴─────────────────┼┴──────────────────┤
│       │                      Docs/Knowledge           │                   │
│       │                            ...                │                   │
│       │                                               │                   │
├───────┼──────────┬──────────────────┬─────────────────┼┬──────────────────┤
│  .─.  └──────.   │  .─.             │                 ││  .─.             │
│ ( x ◀─────( b )  │ ( x )            │                 └┼▶( n )            │
│  `─'       `─'   │  `─'             │                  │  `─'             │
│        95   │    │   │    96        │                  │   │    98        │
│            .▼.   │  .▼.             │                  │   ▼              │
│           ( c )  │ ( d )            │                  │  .─.             │
│            `─'   │  `─'             │                  │ ( x )            │
└──────────────────┴──────────────────┴──────────────────┴──`─'─────────────┘
```

Where, knowledge, the more granular spliting and information with higher density, optionally multi-hop of `x -> y`, `i -> j -> z -> x` etc... across many more nodes(chunks) than K(in TopK search) could be inlucded in Retrievers. And we believe there are cases that this additional work matters.

But how/how well exactly does it work? Let's see in this demo.

# 1. Preparation

## 1.1 Prepare for LLM

In [1]:
# For OpenAI
import os
from llama_index.core import (
    VectorStoreIndex,
    KnowledgeGraphIndex,
    ServiceContext,
)
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import set_global_service_context

from llama_index.core.storage.storage_context import StorageContext
from llama_index.graph_stores.nebula import NebulaGraphStore

from llama_index.llms.openai import OpenAI

import logging
import sys

from IPython.display import Markdown, display

os.environ["OPENAI_API_KEY"] = 


logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
# You need to deploy your own embedding model as well as your own chat completion model
embedding_llm = OpenAIEmbedding(
        model="text-embedding-ada-002",
    )

service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embedding_llm,
)

set_global_service_context(service_context)

/Users/yuxi.zheng/TiHub/inframan/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/var/folders/_p/nbm6_l1d1c9g0kg6tb6c63dw0000gn/T/ipykernel_74825/2188378047.py:35: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


## 1.2. Prepare for Local Graph Store


❗Access local Graph Store Console to **create space** and **graph schema**

```sql
CREATE SPACE guardians(vid_type=FIXED_STRING(256), partition_num=1, replica_factor=1);
:sleep 10;
USE guardians;
CREATE TAG entity(name string);
CREATE EDGE relationship(relationship string);
:sleep 10;
CREATE TAG INDEX entity_index ON entity(name(256));
```

In [2]:
%pip install nebula3-python ipython-ngql

os.environ['NEBULA_USER'] = "root"
os.environ['NEBULA_PASSWORD'] = "nebula" # default password
os.environ['NEBULA_ADDRESS'] = "127.0.0.1:9669" # assumed we have NebulaGraph installed locally

space_name = "guardians"
edge_types, rel_prop_names = ["relationship"], ["relationship"] # default, could be omit if create from an empty kg
tags = ["entity"] # default, could be omit if create from an empty kg

graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)
storage_context = StorageContext.from_defaults(graph_store=graph_store)


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 2. Build the Knowledge Graph

In our demo, the Knowledge Graph was created with LLM.

We simply do so leveragint the `KnowledgeGraphIndex` from LlamaIndex, when creating it, Triplets will be extracted with LLM and evantually persisted into `NebulaGraphStore`.

### 2.1 Preprocess Data

We will download and preprecess data from:
    https://en.wikipedia.org/wiki/Guardians_of_the_Galaxy_Vol._3

In [3]:
from llama_index.core import download_loader

WikipediaReader = download_loader("WikipediaReader")

loader = WikipediaReader()

documents = loader.load_data(pages=['Donald Trump'], auto_suggest=False)

/var/folders/_p/nbm6_l1d1c9g0kg6tb6c63dw0000gn/T/ipykernel_74825/2931829646.py:3: DeprecationWarning: Call to deprecated function (or staticmethod) download_loader. (`download_loader()` is deprecated. Please install tool using pip install directly instead.)
  WikipediaReader = download_loader("WikipediaReader")



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


### 2.2 Extract Triplets and Save to NebulaGraph

This call will take some time, it'll extract entities and relationships and store them into NebulaGraph

In [4]:
kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    service_context=service_context,
    max_triplets_per_chunk=10,
    include_embeddings=True,
)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx

## 3 Create VectorStoreIndex for RAG

To compare with/work together with VectorDB based RAG, let's also create a `VectorStoreIndex`.

During the creation, same data source will be split into chunks and embedding of them will be created, during the RAG query time, the top-k related embeddings will be vector-searched with the embedding of the question.

```
                  RAG with Llama Index
                  ┌────┬────┬────┬────┐                  
                  │ 1  │ 2  │ 3  │ 4  │                  
                  ├────┴────┴────┴────┤                  
                  │  Docs/Knowledge   │                  
┌───────┐         │        ...        │       ┌─────────┐
│       │         ├────┬────┬────┬────┤       │         │
│       │         │ 95 │ 96 │    │    │       │         │
│       │         └────┴────┴────┴────┘       │         │
│ User  │─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─▶   LLM   │
│       │                                     │         │
│       │                                     │         │
└───────┘    ┌ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ┐  └─────────┘
    │          ┌──────────────────────────┐        ▲     
    └────────┼▶│  Tell me ....., please   │├───────┘     
               └──────────────────────────┘              
             │ ┌────┐ ┌────┐               │             
               │ 3  │ │ 96 │                             
             │ └────┘ └────┘               │             
              ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ 
```

In Llama Index, this could be done with oneline of code:

In [5]:
vector_index = VectorStoreIndex.from_documents(
    documents,
    service_context=service_context
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


## 4. Prepare for different query approaches

| QueryEngine | Graph RAG query engine                                       | Vector RAG query engine                                      | Graph RAG with Vector query engine                                                                               |
| ----------- | ------------------------------------------------------------ | ------------------------------------------------------------ |------------------------------------------------------------------------------------------------------------------|
| Mechanism   | 1. Get related entities of the question<br />2. Get n-depth **SubGraphs** of related entities from KG<br />3. Answer synthesis based on related SubGraphs | 1. Create embedding of question<br />2. Semantic search **top-k related doc chunks**<br />3. Answer synthesis based on related doc chunks | 1. Do retrieval as Vector and Graph RAG <br />2. Answer synthesis based on **both related chunks and SubGraphs** |


### 4. Graph RAG query engine

Graph RAG takes SubGraphs related to entities of the task/question as Context.

```
           Graph + Vector RAG with Llama Index
                  ┌────┬────┬────┬────┐                  
                  │ 1  │ 2  │ 3  │ 4  │                  
                  ├────┴────┴────┴────┤                  
                  │  Docs/Knowledge   │                  
┌───────┐         │        ...        │       ┌─────────┐
│       │         ├────┬────┬────┬────┤       │         │
│       │         │ 95 │ 96 │    │    │       │         │
│       │         └────┴────┴────┴────┘       │         │
│ User  │─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─▶   LLM   │
│       │                                     │         │
│       │                                     │         │
└───────┘    ┌ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ┐  └─────────┘
    │          ┌──────────────────────────┐        ▲     
    └────────┼▶│  Tell me about x, please │├───────┘     
               └──────────────────────────┘              
             │ Below are knowledge about x │             
               x->y<-z,x->h->i, m<-n,...                            
             │ Please answer based on them │             
              ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ 
```

In [6]:
kg_rag_query_engine = kg_index.as_query_engine(
    include_text=False,
    retriever_mode="keyword",
    response_mode="tree_summarize",
)

### 4.3 Vector RAG query engine

Vector RAG is the common approach to find topK semantic related doc chunks as context to synthesize the answer.

In [7]:
vector_rag_query_engine = vector_index.as_query_engine()

### 4.4 Graph+Vector RAG query engine

This is a combined Graph+Vector Based RAG, where we will retrieve both VectorDB and KG SubGraphs as the context, for synthesis of the answer.

```
           Graph + Vector RAG with Llama Index
                  ┌────┬────┬────┬────┐                  
                  │ 1  │ 2  │ 3  │ 4  │                  
                  ├────┴────┴────┴────┤                  
                  │  Docs/Knowledge   │                  
┌───────┐         │        ...        │       ┌─────────┐
│       │         ├────┬────┬────┬────┤       │         │
│       │         │ 95 │ 96 │    │    │       │         │
│       │         └────┴────┴────┴────┘       │         │
│ User  │─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─▶   LLM   │
│       │                                     │         │
│       │                                     │         │
└───────┘    ┌ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ┐  └─────────┘
    │          ┌──────────────────────────┐        ▲     
    └────────┼▶│  Tell me ....., please   │├───────┘     
               └──────────────────────────┘              
             │ ┌────┐┌────┐               │             
               │ 3  ││ 96 │ x->y<-z,x->h...                            
             │ └────┘└────┘               │             
              ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ 
```

To implement that in Llama Index, we create a `CustomRetriever` to comebine the two: 

In [8]:
# import QueryBundle
from llama_index.core import QueryBundle

# import NodeWithScore
from llama_index.core.schema import NodeWithScore

# Retrievers
from llama_index.core.retrievers import BaseRetriever, VectorIndexRetriever, KGTableRetriever

from typing import List


class CustomRetriever(BaseRetriever):
    """Custom retriever that performs both Vector search and Knowledge Graph search"""

    def __init__(
        self,
        vector_retriever: VectorIndexRetriever,
        kg_retriever: KGTableRetriever,
        mode: str = "OR",
    ) -> None:
        """Init params."""

        self._vector_retriever = vector_retriever
        self._kg_retriever = kg_retriever
        if mode not in ("AND", "OR"):
            raise ValueError("Invalid mode.")
        self._mode = mode

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""

        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        kg_nodes = self._kg_retriever.retrieve(query_bundle)

        vector_ids = {n.node.node_id for n in vector_nodes}
        kg_ids = {n.node.node_id for n in kg_nodes}

        combined_dict = {n.node.node_id: n for n in vector_nodes}
        combined_dict.update({n.node.node_id: n for n in kg_nodes})

        if self._mode == "AND":
            retrieve_ids = vector_ids.intersection(kg_ids)
        else:
            retrieve_ids = vector_ids.union(kg_ids)

        retrieve_nodes = [combined_dict[rid] for rid in retrieve_ids]
        return retrieve_nodes

Next, we will create instances of the Vector and KG retrievers, which will be used in the instantiation of the Custom Retriever.

In [9]:
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine

# create custom retriever
vector_retriever = VectorIndexRetriever(index=vector_index)
kg_retriever = KGTableRetriever(
    index=kg_index, retriever_mode="keyword", include_text=False
)
custom_retriever = CustomRetriever(vector_retriever, kg_retriever)

# create response synthesizer
response_synthesizer = get_response_synthesizer(
    service_context=service_context,
    response_mode="tree_summarize",
)

/var/folders/_p/nbm6_l1d1c9g0kg6tb6c63dw0000gn/T/ipykernel_74825/1416513360.py:6: DeprecationWarning: Call to deprecated class KGTableRetriever. (KGTableRetriever is deprecated, it is recommended to use PropertyGraphIndex and associated retrievers instead.) -- Deprecated since version 0.10.53.
  kg_retriever = KGTableRetriever(


And the query engine:

In [10]:
graph_vector_rag_query_engine = RetrieverQueryEngine(
    retriever=custom_retriever,
    response_synthesizer=response_synthesizer,
)

## 5. Query with all the Engines

### 5.2 Graph RAG

In [11]:
response_graph_rag = kg_rag_query_engine.query("Tell me about Donald Trump.")

display(Markdown(f"<b>{response_graph_rag}</b>"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


<b>Donald Trump has been involved in various relationships and actions such as being criticized by the World Health Organization, causing an increased incidence of hate crimes, being alleged of election fraud, filing for Chapter 11 bankruptcy protection, playing different versions of himself, and promoting conspiracy theories.</b>

### 5.3 Vector RAG

In [12]:
response_vector_rag = vector_rag_query_engine.query("Tell me about Donald Trump.")

display(Markdown(f"<b>{response_vector_rag}</b>"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


<b>Donald Trump is an American politician, media personality, and businessman who served as the 45th president of the United States from 2017 to 2021. He has a Bachelor of Science in economics from the University of Pennsylvania and is known for his real estate ventures, including building skyscrapers, hotels, casinos, and golf courses. Trump has been involved in numerous legal actions and business bankruptcies. As president, he implemented policies such as a travel ban on citizens from certain countries, diverting military funding for a border wall, and rolling back environmental regulations. Trump has been impeached twice, faced controversies over his handling of the COVID-19 pandemic, and made false statements during his presidency. He continues to be a prominent figure in the Republican Party and has faced legal issues post-presidency, including being found guilty of falsifying business records and indicted on multiple felony counts.</b>

In [13]:
from IPython.display import Markdown, display
from llama_index.llms.openai import OpenAI
from llama_index.core.prompts.prompts import PromptTemplate

llm = OpenAI(temperature=0, model="gpt-3.5-turbo")

prompt_template = PromptTemplate(
    input_variables="graph_result, vector_result",
    template="""
Compare the two QA results on "Tell me about Donald Trump." List the differences between them to help evaluate them. Output in a markdown table.

**Result from Graph:**
{graph_result}

---

**Result from Vector:**
{vector_result}
"""
)

prompt_args = {
    "graph_result": response_graph_rag,
    "vector_result": response_vector_rag
}

comparison_result = llm.predict(prompt_template, **prompt_args)

# Display the comparison result as Markdown
display(Markdown(comparison_result))


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


| Aspect           | Graph Result                                                                                                                                                                                                                                      | Vector Result                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
|------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Background       | Focuses on relationships, criticisms, hate crimes, election fraud, bankruptcy, playing different versions of himself, promoting conspiracy theories                                                                                               | Mentions he is an American politician, media personality, businessman, served as the 45th president of the United States, has a Bachelor of Science in economics, known for real estate ventures, legal actions, business bankruptcies, implemented policies as president, impeached twice, controversies over COVID-19 handling, false statements, prominent figure in Republican Party, legal issues post-presidency |
| Achievements     | Does not mention any specific achievements                                                                                                                                                                                                        | Mentions building skyscrapers, hotels, casinos, golf courses                                                                                                                                                                                                                                                                                                                                                                                                                                           |
| Policies         | Does not mention any specific policies                                                                                                                                                                                                            | Mentions policies such as travel ban, diverting military funding for border wall, rolling back environmental regulations                                                                                                                                                                                                                                                                                                                                                                             |
| Legal Issues     | Mentions being criticized by WHO, alleged election fraud, filing for bankruptcy protection                                                                                                                                                        | Mentions impeachments, controversies over COVID-19 handling, false statements, legal issues post-presidency                                                                                                                                                                                                                                                                                                                                                                                         |
| Post-Presidency  | Does not mention any post-presidency information                                                                                                                                                                                                  | Mentions being a prominent figure in Republican Party, facing legal issues post-presidency, being found guilty of falsifying business records, indicted on multiple felony counts                                                                                                                                                                                                                                                                                                                     |

### 5.4 Graph + Vector RAG

In [14]:
response_graph_vector_rag = graph_vector_rag_query_engine.query("Tell me about Donald Trump.")

display(Markdown(f"<b>{response_graph_vector_rag}</b>"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


<b>Donald Trump was born on June 14, 1946, in Queens, New York City. He attended the private Kew-Forest School and later the New York Military Academy before studying economics at the Wharton School of the University of Pennsylvania. Trump is a businessman, media personality, and politician who served as the 45th president of the United States from 2017 to 2021. He has been involved in various business ventures, including real estate, casinos, and television. Throughout his presidency, Trump implemented policies such as tax cuts, border security measures, and changes to environmental regulations. He faced impeachment twice during his presidency and has been involved in numerous legal actions. Trump continues to be a prominent figure in the Republican Party and is a candidate in the 2024 presidential election.</b>

## 6. Conclusion

### 6.1 Overall Comparision

Let's compare the results of them.

First check the information that were coverred by different approaches:

In [15]:
from llama_index.core.prompts.prompts import PromptTemplate
from IPython.display import Markdown, display

prompt_template = PromptTemplate(
    input_variables="response_graph_rag, response_vector_rag, response_graph_vector_rag",
    template="""
Compare the QA results on "Tell me about Donald Trump.", list the knowledge facts between them, to help evaluate them. Output in markdown table.

Result Graph: {response_graph_rag}
---
Result Vector: {response_vector_rag}
---
Result Graph+Vector: {response_graph_vector_rag}
---
"""
)

prompt_args = {
    "response_graph_rag": response_graph_rag,
    "response_vector_rag": response_vector_rag,
    "response_graph_vector_rag": response_graph_vector_rag
}

comparison_result = llm.predict(prompt_template, **prompt_args)

display(Markdown(comparison_result))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


| Knowledge Fact                                      | Result Graph | Result Vector | Result Graph+Vector |
|-----------------------------------------------------|--------------|---------------|---------------------|
| Born on June 14, 1946 in Queens, New York City      | ✓            | ✓             | ✓                   |
| Attended Wharton School of the University of Pennsylvania | ✗            | ✓             | ✓                   |
| Bachelor of Science in economics from UPenn         | ✗            | ✓             | ✓                   |
| Businessman, media personality, politician          | ✗            | ✓             | ✓                   |
| 45th president of the United States (2017-2021)     | ✗            | ✓             | ✓                   |
| Involved in various business ventures               | ✗            | ✗             | ✓                   |
| Implemented policies like tax cuts, border security | ✗            | ✗             | ✓                   |
| Faced impeachment twice during presidency           | ✗            | ✗             | ✓                   |
| Involved in numerous legal actions and bankruptcies | ✓            | ✓             | ✓                   |
| Prominent figure in the Republican Party            | ✗            | ✓             | ✓                   |
| Candidate in the 2024 presidential election         | ✗            | ✗             | ✓                   |
| Controversies over handling of COVID-19 pandemic    | ✗            | ✗             | ✓                   |
| Made false statements during presidency             | ✗            | ✗             | ✓                   |
| Legal issues post-presidency                        | ✗            | ✓             | ✓                   |

**Conclusion**

For the text scenario( need LLM extract to build a KG):

- The information loss during the extraction of building KG could impact the GraphRAG peformance a lot 
- The Graph RAG comes with **concise** results, and much **lower cost** (for cost comparison see  result [here](https://gpt-index.readthedocs.io/en/latest/examples/index_structs/knowledge_graph/KnowledgeGraphIndex_vs_VectorStoreIndex_vs_CustomIndex_combined.html#comparison-of-results))
- The Vector RAG can return fruitful answer, maybe less comprehensive.
- The **Graph+Vector** RAG could be more **comprehensive** in case the question involves knowledge that's fine-grained **spread** across more chunks than top-K searching.

| QueryEngine | Graph RAG query engine                                       | Vector RAG query engine                                      | Graph Vector RAG query engine                                |
| ----------- | ------------------------------------------------------------ | ------------------------------------------------------------ | ------------------------------------------------------------ |
| Mechanism   | 1. Get related entities of the question<br />2. Get n-depth **SubGraphs** of related entities from KG<br />3. Answer synthesis based on related SubGraphs | 1. Create embedding of question<br />2. Semantic search **top-k related doc chunks**<br />3. Answer synthesis based on related doc chunks | 1. Do retrieval as Vector and Graph RAG <br />2. Answer synthesis based on **both related chunks and SubGraphs** |
| Performance | Concise                                                      | Fruitful                                                     | Fruitful, could be more comprehensive                        |
| Cost        | Low                                                          | High                                                         | High                                                         |
